# HOSPITAL to OMOP CDM


In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

This mapping is very simple, and following https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=1871150220#gid=1871150220 it is just a care site with as name the one in IDEA.

In [ ]:
df_hospital_IDEA4RC = pd.read_csv("./IDEA4RC-data/hospitalDataIDEA4RC.csv")
df_hospital_IDEA4RC.head(5)

In [ ]:
conn_params = {
    'dbname': 'omopdb',
    'user': 'postgres',
    'password': 'mysecretpassword',
    'host': 'omop_postgres',
    'port': '5432'
}

try:
    conn = psycopg2.connect(**conn_params)
    cur = conn.cursor()
    
    cur.execute("SELECT current_database();")
    current_db = cur.fetchone()[0]
    print("Base de datos actual:", current_db)
    
    cur.execute("SELECT table_schema, table_name FROM information_schema.tables WHERE table_name = 'care_site';")
    tables = cur.fetchall()
    if not tables:
        print("Table does not exist.")    
except Exception as e:
    print(e)

### From Hospital to Care Site

In [ ]:
hospital_mapping = 'Hospital Name'
df_table = pd.DataFrame()
df_table.index = range(len(df_hospital_IDEA4RC))
sql = """
    INSERT INTO care_site (care_site_name)
    VALUES (%s)
"""

cur.executemany(sql, zip(df_hospital_IDEA4RC['Hospital Name']))
conn.commit()


In [ ]:
cur.close()
conn.close()